Force de gravitation (Terre) :
$$\ddot r = -\frac{GM}{\mathbf{r}^3} \mathbf{r}$$

Influence du soleil :
$$\ddot r = \frac{GM\mathbf r}{s^3}\big(-\mathbf{e}_r+3\mathbf{e}_s(\mathbf{e}_s\mathbf{e}_r)\big)$$

Influence de la lune :
$$\ddot r = \frac{GM\mathbf r}{l^3}\big(-\mathbf{e}_r+3\mathbf{e}_l(\mathbf{e}_l\mathbf{e}_r)\big)$$

Trainée atmosphérique :
$$ \ddot r = -\frac12 C_D \frac{A}m \rho_0 e^{-h/H_0} \left(\dot r - \omega r\right)^2 \textbf e_v $$

Géopotentiel :
$$
\ddot{r} = \nabla\frac{GM}{r} + \nabla\frac{GM}{r} \frac{R^2}{r^2} \sqrt{5} \frac{1}{2}\left(3 \sin^2\varphi - 1\right) \bar{C}_{2 ,0} + \nabla\frac{GM}{r}\frac{R^2}{r^2} 3\cos^2\varphi \sqrt \frac{5}{12} \left(\bar{C}_{2,2} \cos(2\lambda) + \bar{S}_{2,2} \sin(2\lambda)\right)
$$

Simplification de la formule :
$$
\ddot r = \nabla\frac{GM}r\left[1+\frac{\sqrt5}2\frac{R^2}{r^2}\Bigg(\bar{C}_{2,0}\left(3\sin^2(\varphi)-1\right)+\sqrt3\cos^2(\varphi)\left(\bar{C}_{2,2}\cos(2\lambda)+\bar{S}_{2,2}\sin(2\lambda)\right)\Bigg)\right]
$$

Suppression (arbitraire) du facteur 1 :
$$
\ddot r = \nabla\frac{GM}r\frac{\sqrt5}2\frac{R^2}{r^2}\Bigg(\bar{C}_{2,0}\left(3\sin^2(\varphi)-1\right)+\sqrt3\cos^2(\varphi)\left(\bar{C}_{2,2}\cos(2\lambda)+\bar{S}_{2,2}\sin(2\lambda)\right)\Bigg)
$$

In [1]:
from abc import ABC, abstractmethod
import numpy as np

class Force(ABC):
    def __init__(self):
        pass

    @property
    def G(self):
        """Constance gravitationnelle."""
        return 6.67430e-20  # km3 kg^−1 s^−2

    @property
    def M(self):
        """Masse de la Terre."""
        return 5.9736e24  # kg

    @property
    def R(self):
        """Rayon moyen volumétrique de la Terre."""
        return 6371  # km

    @property
    @abstractmethod
    def name(self):
        pass
    
    @property
    @abstractmethod
    def value(self):
        pass
    
    @abstractmethod
    def acceleration(self, u, t):
        raise NotImplementedError(self.acceleration)
        
    def acc_norm(self, u, t=0):
        a = self.acceleration(u, t)
        return np.linalg.norm(a)
    
    def __repr__(self):
        return self.name
    
    def _repr_latex_(self):
        return fr"{self.name} : $\displaystyle {self.value}$"


class ForceSet():
    def __init__(self, forces):
        self.forces = forces

    def derivee(self, u, t):
        # vecteur des coordonnées sources
        x, y, z, vx, vy, vz = u
        # les vitesses sont les dérivées des positions
        dx, dy, dz = vx, vy, vz
        # les acclélérations sont définies par les EDO de chaque force
        dvx, dvy, dvz = np.sum([force.acceleration(u, t) for force in self.forces], axis=0)
        # renvoi des dérivées
        return np.array([dx, dy, dz, dvx, dvy, dvz])

    def __repr__(self):
        return fr"ForceSet[{', '.join(str(force) for force in self.forces)}]"    

In [2]:
def ed_rk4(start, end, step, init, derivee, ordre=1):
    """Résolution d'équation différentielle par la méthode de Runge-Kutta.

    Attributs:
      - start: valeur de départ de la variable temporelle
      - end: valeur d'arrivée de la variable temporelle
      - init: valeur initiale de la variable libre
      - derivee: fonction dérivée recevant en paramètre les variables libre et temporelle
      - ordre: ordre de l'équation différentielle (1 par défaut)
    """
    # Création du tableau temps
    num_points = int((end - start) / step) + 1
    t = np.linspace(start, end, num_points)

    # Initialisation du tableau solution
    v = np.empty((ordre, num_points))

    # Condition initiale
    v[:, 0] = init

    # Boucle for des variables de la méthode
    for i in range(num_points - 1):
        d1 = derivee(v[:, i], t[i])
        d2 = derivee(v[:, i] + step / 2 * d1, t[i] + step / 2)
        d3 = derivee(v[:, i] + step / 2 * d2, t[i] + step / 2)
        d4 = derivee(v[:, i] + step * d3,  t[i] + step)
        v[:, i + 1] = v[:, i] + step / 6 * (d1 + 2*d2 + 2*d3 + d4)

    return t, v

In [3]:
class Earth_Gravity(Force):

    @property
    def name(self):
        return "Force de gravitation terrestre"
    
    @property
    def value(self):
        return r"-\frac{GMm}{r^3}\mathbf r"

    def acceleration(self, u, t):
        """EDO gravitation terrestre."""
        # vecteur position        
        r = np.array(u[:3])
        # Expression de l'acceleration résultante de la force
        return - self.G * self.M / np.linalg.norm(r)**3 * r

In [4]:
from astropy.time import Time
import astropy.units as units


class Body_Influence(Force):
    def __init__(self, body_name, astropy_position_callback, t0):
        self.body_name = body_name
        self.pos = astropy_position_callback
        self.t0 = Time(iss_data.iloc[0].datetime)

    @property
    def name(self):
        return f"Influence gravitationnelle relative : {self.body_name}"
    
    @property
    def value(self):
        b = self.body_name[0].lower()
        return fr"-\frac{{GMm}}{{{b}^3}}\big(-\mathbf e_r + 3\mathbf e_{b}(\mathbf e_{b}\mathbf e_r)\big)"

    def position_at(self, t):
        time = self.t0 + t * units.s
        return self.pos(time).represent_as('cartesian').xyz.to(units.km).value

    def acceleration(self, u, t):
        """EDO Influence relative d'un corps."""
        # vecteur position        
        r = np.array(u[:3])
        b = self.position_at(t)
        er = r / np.linalg.norm(r)
        eb = b / np.linalg.norm(b)
        # Expression de l'acceleration résultante de la force
        return self.G * self.M * r / np.linalg.norm(b)**3 * (- er + 3 * eb * np.dot(eb, er))

In [5]:
class Atmospheric_Drag(Force):
    def __init__(self, mass, drag_coeff, drag_area):
        self.mass = mass
        self.drag_coeff = drag_coeff
        self.drag_area = drag_area

    @property
    def name(self):
        return "Traînée atmosphérique"
    
    @property
    def value(self):
        return r"-\frac12 C_D \frac{A}m \rho_0 e^{-h/H_0} \left(\dot r - \omega r\right)^2 \textbf e_v"

    @property
    def rho_0(self):
        """Densité atmosphérique du point de référence, ici le niveau de la mer."""
        return 1.225  # kg.m^-3
    
    @property
    def H_0(self):
        """Hauteur de l'échelle de densité, ici la hauteur moyenne de la mer."""
        return 7.9  # km

    @property
    def omega(self):
        """Vitesse angulaire moyenne de la Terre."""
        return 7.292e-5 # rad.s^-1

    def acceleration(self, u, t):
        """EDO traînée atmosphérique."""
        # vecteurs position et vitesse        
        r, dotr = np.split(np.array(u), 2)
        # Hauteur de la station
        h = np.linalg.norm(r) - self.R
        # Vitesse relative à celle de l'atmosphère
        v_r = (dotr - self.omega * r)
        # Vecteur unitaire de v_r
        e_v = v_r / np.linalg.norm(v_r)
        # Expression de l'acceleration résultante de la force
        a = -1/2 * self.drag_coeff * self.drag_area/self.mass * self.rho_0 * np.exp(-h/self.H_0) * v_r.dot(v_r) * e_v
        return a

In [6]:
from astropy.coordinates import SkyCoord, GCRS

class Geopotential(Force):

    @property
    def C20(self):
        """Valeur du coefficient de Legendre zonal cosinus de degré 2 et d'ordre 0."""
        return -484.2e-6

    @property
    def C22(self):
        """Valeur du coefficient de Legendre cosinus de degré 2 et d'ordre 2."""
        return 2.439261e-6

    @property
    def S22(self):
        """Valeur du coefficient de Legendre sinus de degré 2 et d'ordre 2."""
        return -1.400266e-6

    @property
    def name(self):
        return "Force géopotentielle"
    
    @property
    def value(self):
        return (r"\nabla\frac{GM}r\left[1+\frac{\sqrt5}2\frac{R^2}{r^2}"
                r"\Bigg(\bar{C}_{2,0}\left(3\sin^2(\varphi)-1\right)+"
                r"\sqrt3\cos^2(\varphi)\left(\bar{C}_{2,2}\cos(2\lambda)+"
                r"\bar{S}_{2,2}\sin(2\lambda)\right)\Bigg)\right]")

    def acceleration(self, u, t):
        """EDO géopotentiel."""
        # vecteur position        
        r = np.array(u[:3])
        # longitude et latitude stellaire
        sk = SkyCoord(*r, unit=units.km, frame=GCRS, representation_type='cartesian').represent_as('spherical')
        longitude, latitude = sk.lon.value, sk.lat.value
        # Norme de R
        n_r = sk.distance.value  # = np.linalg.norm(r)
        # nabla(GM/r)
        nablaGMr = - self.G *self.M / n_r**3 * r
        # Expression de l'acceleration résultante de la force
        a = nablaGMr * (np.sqrt(5)*self.R**2/(2*n_r**2)
                            * (self.C20 * (3 * np.sin(latitude)**2 - 1)
                               + np.sqrt(3) * np.cos(latitude)**2
                               * (self.C22 * np.cos(2*longitude) + self.S22 * np.sin(2*longitude))))
        return a

In [7]:
import isslib

# Coordonnées de l'ISS
iss = isslib.ISS_Position()
iss_data = iss.get_data().query('thrust_episode==0')
# Métadonnées utiles
iss_mass = iss.get_metadata("MASS")
iss_drag_coeff = iss.get_metadata("DRAG_COEFF")
iss_drag_area = iss.get_metadata("DRAG_AREA")
# Colonnes d'interêt
columns = ['x', 'y', 'z', 'vx', 'vy', 'vz']
# Variables initiales
ini = iss_data.iloc[0][columns]
t0 = iss_data.iloc[0]['datetime']
# Durée de la plage temporelle
t_final = iss_data.iloc[-1]['datetime']
duration = (t_final - t0).total_seconds()
# Etapes de 4 minutes (identique aux données sources)
t_step = 240

In [8]:
import astropy.coordinates as co

forces = [Earth_Gravity(),
          Geopotential(),
          Body_Influence('Lune',co.get_moon, t0),
          Body_Influence('Soleil',co.get_sun, t0),
          Atmospheric_Drag(iss_mass, iss_drag_coeff, iss_drag_area),
         ]
for force in forces:
    print(force, ':', np.log10(force.acc_norm(ini)))

Force de gravitation terrestre : -2.06384902291244
Force géopotentielle : -5.711414564037035
Influence gravitationnelle relative : Lune : -7.460468435457468
Influence gravitationnelle relative : Soleil : -15.285847740624178
Traînée atmosphérique : -23.898055719710644


In [9]:
import pandas as pd

def model(coords, forces, t_start, t_stop, t_step):
    columns = coords.index if isinstance(coords, pd.Series) else None
    forceset = ForceSet(forces)
    t, drk4 = ed_rk4(t_start, t_stop, t_step, coords, forceset.derivee, len(coords))
    return pd.DataFrame(drk4.T, index=t, columns=columns)

duration = 681000  # Cap for debug
df = model(ini, forces, 0, duration, t_step)


In [38]:
df  # wtf?

,x,y,z,vx,vy,vz
0.0,-3.149866e+03,3.681707e+03,-4.765218e+03,-3.379971e+00,-6.332376e+00,-2.663563e+00
240.0,-3.836670e+03,2.046638e+03,-5.223464e+03,-2.308895e+00,-7.210562e+00,-1.132254e+00
480.0,-4.244637e+03,2.628290e+02,-5.302090e+03,-1.070374e+00,-7.564491e+00,4.808461e-01
720.0,-4.344172e+03,-1.540069e+03,-4.995450e+03,2.458684e-01,-7.368727e+00,2.058929e+00
960.0,-4.128012e+03,-3.231036e+03,-4.325795e+03,1.544542e+00,-6.637346e+00,3.487726e+00
...,...,...,...,...,...,...
679920.0,1.414547e+03,-5.487654e+03,3.053396e+03,3.868558e+00,3.617766e+00,3.913401e+00
680160.0,-5.853806e+06,-8.421028e+06,-5.219666e+06,5.429818e+15,7.803943e+15,4.843313e+15
680400.0,1.303156e+18,1.872946e+18,1.162395e+18,5.429817e+15,7.803944e+15,4.843313e+15
680640.0,2.606312e+18,3.745893e+18,2.324790e+18,5.429813e+15,7.803945e+15,4.843313e+15


In [39]:
df2 = df[df.index <= 680160]
with isslib.orbital_visualization() as fig:
    fig.add_scatter3d(x=df2.x, y=df2.y, z=df2.z, line_color='red', name="Episode 0")
    fig.update_layout(title="Modélisation de gravitation terrestre à partir de coordonnées initiales")